<a href="https://colab.research.google.com/github/aliffaturrohman/fake-news-classification-with-granite/blob/main/Fake_News_Classiffication_with_IBM_Granite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fake News Classification with IBM Granite

  ![Fake News](https://storage.googleapis.com/kaggle-datasets-images/3950844/6875518/662116dc40ef78f6d3a65355233cb215/dataset-cover.JPG?t=2023-11-03-17-46-02)

![Static Badge](https://img.shields.io/badge/Python-3.10-blue)
![Static Badge](https://img.shields.io/badge/Jupyter-Notebook-orange)
![Static Badge](https://img.shields.io/badge/-Machine%20Learning-orange)
![Static Badge](https://img.shields.io/badge/-NLP-yellowgreen)

In [ ]:
!curl -L -o ~/Downloads/fake-news.zip\
  https://www.kaggle.com/api/v1/datasets/download/rajatkumar30/fake-news

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 11.4M    0  1177    0     0   1420      0  2:21:02 --:--:--  2:21:02  1420
curl: (23) Failure writing output to destination


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import zipfile
import os

zip_file_path = 'news.csv.zip'
extract_dir = '.'  # Extract to the current directory

try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Successfully extracted {zip_file_path} to {extract_dir}")
except FileNotFoundError:
    print(f"Error: The file {zip_file_path} was not found.")
except zipfile.BadZipFile:
    print(f"Error: The file {zip_file_path} is not a valid zip file.")
except Exception as e:
    print(f"An error occurred: {e}")

Successfully extracted news.csv.zip to .


In [ ]:
df = pd.read_csv("news.csv.zip")
df.drop(columns=['Unnamed: 0'], inplace=True)
df = df.loc[:1999]

In [ ]:
df.head()

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [ ]:
df.shape

(2000, 3)

In [ ]:
!pip install colab-xterm -q
!curl https://ollama.ai/install.sh | sh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 8.2 MB/s eta 0:00:00
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13281    0 13281    0     0  35597      0 --:--:-- --:--:-- --:--:-- 35605
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
%load_ext colabxterm
%xterm
# in the command console, type this
# ollama serve & ollama pull granite3.3:8b & ollama run granite3.3:8b
# ollama serve & ollama run granite3.3:8b

ModuleNotFoundError: No module named 'colabxterm'

In [ ]:
!pip install langchain langchain-ollama -q
!pip install langchain-community langchain-core -q


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -1


In [ ]:
from langchain.prompts import PromptTemplate

template = """
You are a strict classifier. Your job is to read the news article below and classify it into ONLY ONE WORD: either "REAL" or "FAKE".

Do NOT provide any reasoning or explanation. Do NOT add any additional words, symbols, or punctuation.
Your response must be exactly one of these two words:
- REAL
- FAKE

Classification Criteria:
REAL News typically includes:
- Neutral or factual headlines
- Recognized news sources (e.g., *Reuters*, *BBC*, *The Washington Post*)
- Specific, verifiable information (data, quotes, evidence)
- Formal and objective language

FAKE News typically includes:
- Sensational or emotionally charged headlines
- Dubious or unnamed sources (e.g., *Before It’s News*, anonymous blogs)
- Vague or unverifiable claims
- Exaggerated or conspiratorial language

Examples:
1. REAL → Headline: "100,000 protest in the capital", Source: *The Washington Post*
2. FAKE → Headline: "Breaking: Government Hides the Truth About Aliens!", Source: *Before It's News*

Article:
Headline: {headline}
Text: {text}

Respond with only one word: REAL or FAKE
Your answer:
"""

prompt = PromptTemplate(
    input_variables=["headline", "text"],
    template=template
)

In [ ]:
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnableLambda

llm = OllamaLLM(model="granite3.3:8b")

chain = RunnableLambda(lambda inputs: llm.invoke(prompt.format(**inputs)))

# Testing model Granite
headline = df.loc[0, 'title']
text = df.loc[0, 'text']
response = chain.invoke({"headline": headline, "text": text})
print(response)

FAKE


In [ ]:
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnableLambda

llm = OllamaLLM(model="granite3.3:8b")

chain = RunnableLambda(lambda inputs: llm.invoke(prompt.format(**inputs)))

label_response = []

for index, row in df.iterrows():
    headline = row['title']
    text = row['text']
    response = chain.invoke({"headline": headline, "text": text})
    progress = (index + 1) / len(df) * 100
    print(f"{index} | Response: {response} | Progress: {progress:.2f}%")
    label_response.append(response)


ConnectError: [Errno 111] Connection refused

In [ ]:
import re

def extract_label(response_list):
    labels = []
    for response in response_list:
        match = re.search(r"\b(REAL|FAKE)\b", response, re.IGNORECASE)
        if match:
            labels.append(match.group(1).upper())
        else:
            labels.append("UNKNOWN")
    return labels


clean_labels = extract_label(label_response)
print(clean_labels)


['REAL', 'FAKE', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'FAKE', 'FAKE', 'REAL', 'REAL', 'FAKE', 'FAKE', 'REAL', 'REAL', 'REAL', 'FAKE', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'FAKE', 'FAKE', 'REAL', 'REAL', 'FAKE', 'FAKE', 'REAL', 'FAKE', 'FAKE', 'FAKE', 'REAL', 'REAL', 'FAKE', 'FAKE', 'REAL', 'REAL', 'REAL', 'REAL', 'FAKE', 'FAKE', 'REAL', 'REAL', 'REAL', 'FAKE', 'REAL', 'REAL', 'REAL', 'REAL', 'FAKE', 'REAL', 'REAL', 'FAKE', 'REAL', 'FAKE', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'FAKE', 'REAL', 'FAKE', 'REAL', 'REAL', 'FAKE', 'REAL', 'REAL', 'REAL', 'REAL', 'FAKE', 'REAL', 'REAL', 'REAL', 'FAKE', 'REAL', 'FAKE', 'REAL', 'FAKE', 'REAL', 'REAL', 'REAL', 'FAKE', 'FAKE', 'REAL', 'REAL', 'FAKE', 'FAKE', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'FAKE', 'FAKE', 'FAKE', 'FAKE', 'REAL', 'REAL', 'FAKE', 'REAL', 'FAKE', 'REAL', 'REAL', 'FAKE', 'REAL', 'REAL', 'REAL', 'FAKE', 'REAL', 'FAKE', 'REAL',

In [ ]:
val_label = (df.label).to_numpy()
pred_label = clean_labels

In [ ]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(val_label, pred_label, labels=["REAL", "FAKE"])
print(conf_matrix)


[[473  26]
 [237 264]]


In [ ]:
from sklearn.metrics import classification_report

report = classification_report(val_label, pred_label, labels=["REAL", "FAKE"])
print(report)


              precision    recall  f1-score   support

        REAL       0.67      0.95      0.78       499
        FAKE       0.91      0.53      0.67       501

    accuracy                           0.74      1000
   macro avg       0.79      0.74      0.72      1000
weighted avg       0.79      0.74      0.72      1000

